# Defining Tools

In [ ]:
from guidance import Tool
from typing import Literal
import random

def get_weather(city: str, unit: Literal["celsius", "fahrenheit"] = "celsius") -> str:
    """
    Get the current weather for a given city.
    
    Args:
        city (str): The name of the city to get the weather for.
        unit (Literal["celsius", "fahrenheit"]): The unit of temperature to return.
    
    Returns: 
        str: A string describing the current weather in the specified city.
    """
    temp = random.randint(-10, 35)
    if temp <= 0:
        weather = "snowy"
    elif temp <= 20:
        weather = "cloudy"
    else:
        weather = "sunny"
    if unit == "fahrenheit":
        temp = temp * 9/5 + 32
    return f"The current temperature in {city} is {temp} degrees {unit} and it is {weather}."

A `Tool` consist of:
1. A `callable` that should be invoked when the tool is called
2. A `name` that uniquely identifies the callable
3. A `description` that indicates the usage of the callable
4. A `schema` that specifies the types of the callable's arguments, along with any metadata such as examples, default values, etc.
    - This `schema` may be a JSON schema (`dict[str, Any]`) or a `pydantic.BaseModel`


In [2]:
from pydantic import BaseModel, Field, ConfigDict

class GetWeatherArgs(BaseModel):
    city: str = Field(..., description="The name of the city to get the weather for.")
    unit: Literal["celsius", "fahrenheit"] = Field("celsius", description="The unit of temperature to return.")

    model_config = ConfigDict(
        extra="forbid", # Disallow extra fields -- required for OpenAI
    )

tool = Tool(
    callable=get_weather,
    name="get_weather",
    description="Get the current weather for a given city.",
    schema=GetWeatherArgs
)

print(f"""\
Tool name:
{tool.name}

Tool description:
{tool.description}

Tool schema:
{tool.schema_dump()}\
""")

Tool name:
get_weather

Tool description:
Get the current weather for a given city.

Tool schema:
{'additionalProperties': False, 'properties': {'city': {'description': 'The name of the city to get the weather for.', 'title': 'City', 'type': 'string'}, 'unit': {'default': 'celsius', 'description': 'The unit of temperature to return.', 'enum': ['celsius', 'fahrenheit'], 'title': 'Unit', 'type': 'string'}}, 'required': ['city'], 'title': 'GetWeatherArgs', 'type': 'object'}


As an alternative to constructing a `Tool` manually, `guidance` can infer the tool's `name`, `description`, and `schema` from the callable's name, docstring, and signature annotations, respectively.

In [24]:
tool = Tool.from_callable(get_weather)

print(f"""\
Tool name:
{tool.name}

Tool description:
{tool.description}

Tool schema:
{tool.schema_dump()}\
""")

Tool name:
get_weather

Tool description:
Get the current weather for a given city.
    
    Args:
        city (str): The name of the city to get the weather for.
        unit (Literal["celsius", "fahrenheit"]): The unit of temperature to return.
    
    Returns: 
        str: A string describing the current weather in the specified city.

Tool schema:
{'additionalProperties': False, 'properties': {'city': {'title': 'City', 'type': 'string'}, 'unit': {'enum': ['celsius', 'fahrenheit'], 'title': 'Unit', 'type': 'string'}}, 'required': ['city', 'unit'], 'title': 'get_weather', 'type': 'object'}


# Calling Tools

## Remote Models

Most remote models have native support for tool-calling.

With these models, you can simply pass a list of tools to `gen`.

Behind the scenes, `guidance` automatically translates the `gen` to a call to the model provider's API, which in the case of OpenAI looks like:

```python
from openai import OpenAI
client = OpenAI(...)

completion = client.chat.completions.create(
    messages=[...],
    tools=[
        {
            "type": "function",
            "function": {
                "name": tool.name,
                "description": tool.description,
                "parameters": tool.schema.model_json_schema(),
                "strict": True,
            }
        },
        ...
    ]
)
```

*Note*: you may pass a list of `callable` instead, and the `Tool`s will be automatically constructed for you.

In [ ]:
from guidance.models import OpenAI
from guidance import user, assistant, gen
lm = OpenAI("gpt-4o-mini")

with user():
    lm += "What is the weather like in San Francisco?"
with assistant():
    lm += gen(tools=[tool])
with assistant():
    lm += gen()

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

You may notice that the tool call is wrapped in `<function={name}>...</function>` tags and the result is wrapped in `<function_result>..</function_result>` tags. This is purely for visualization purposes, as OpenAI uses a structured representation under the hood which we (somewhat arbitrarily) represent as a string above.

## Local Models

In [ ]:
from guidance import ToolCallHandler, json as gen_json
from guidance.types import Grammar
from typing import Any
import re
from json import dumps, loads


class CustomHandler(ToolCallHandler):
    def trigger(self):
        return "<function"

    def begin(self, tool_name: str) -> str:
        return f"<function={tool_name}>"

    def body(self, tool: Tool) -> Grammar:
        return gen_json(schema=tool.schema.model_json_schema())

    def end(self) -> str:
        return "</function>\n"

    def parse_tool_calls(self, text: str) -> tuple[str, dict[str, Any]]:
        matches = re.finditer(
            r"<function=(?P<name>[^>]+)>(?P<args>\{(.|\n)*\})</function>",
            text
        )
        tool_calls = []
        for match in matches:
            tool_calls.append((match.group("name"), loads(match.group("args"))))
        return tool_calls

    def format_return_value(self, value: Any) -> str:
        return f"<function_result>{dumps(value)}</function_result>"

In [ ]:
from guidance.models import LlamaCpp
from guidance import system, user, assistant, gen
from huggingface_hub import hf_hub_download
from json import dumps

lm = LlamaCpp(
    hf_hub_download(
        repo_id="bartowski/Llama-3.2-3B-Instruct-GGUF",
        filename="Llama-3.2-3B-Instruct-Q6_K_L.gguf",
    ),
    n_ctx=4096,
    tool_call_handler_cls=CustomHandler
)

with system():
    lm += f"""\
# Tool Instructions
You have access to the following functions:
{
    dumps(
        [
            {"name": tool.name, "description": tool.description, "arguments": tool.schema_dump()}
        ]
    )
}

If you choose to call a function, you must return the function name and arguments in the following format:
<function={{function_name}}>{{parameters}}</function>
where {{function_name}} is the name of the function you are calling, and {{parameters}} is a JSON object containing the parameters for the function call.

The return value of the function call will be provided to you in the following format:
<function_result>{{return_value}}</function_result>

After the return value is provided, you should summarize the information and provide a final response.
"""
with user():
    lm += "What is the weather like in San Francisco?"
with assistant():
    lm  += gen(tools=[tool]) 
with assistant():
    lm  += gen()

llama_context: n_ctx_per_seq (4096) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_set_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_c4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f16                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

In [30]:
print(str(lm))

<|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024# Tool Instructions
You have access to the following functions:
[{"name": "get_weather", "description": "Get the current weather for a given city.\n    \n    Args:\n        city (str): The name of the city to get the weather for.\n        unit (Literal[\"celsius\", \"fahrenheit\"]): The unit of temperature to return.\n    \n    Returns: \n        str: A string describing the current weather in the specified city.", "arguments": {"additionalProperties": false, "properties": {"city": {"title": "City", "type": "string"}, "unit": {"enum": ["celsius", "fahrenheit"], "title": "Unit", "type": "string"}}, "required": ["city", "unit"], "title": "get_weather", "type": "object"}}]

If you choose to call a function, you must return the function name and arguments in the following format:
<function={function_name}>{parameters}</function>
where {function_name} is the name of the function you are 